In [1]:
import cv2
import numpy as np

hw1_input_video = cv2.VideoCapture('homework_1_test_video.mp4')  #輸入的影片
hw1_output_video = cv2.VideoWriter('myHW1Video.mp4', cv2.VideoWriter_fourcc(*'XVID'),
                                   hw1_input_video.get(cv2.CAP_PROP_FPS),
                                   (int(hw1_input_video.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                    int(hw1_input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))))  #輸出的影片

# 設定基本資訊

In [2]:
frame_num = 0
total_frame = int(hw1_input_video.get(cv2.CAP_PROP_FRAME_COUNT))


## 設定是第幾個frame
def set_frame_number(x):
    global frame_num
    frame_num = x
    return


cv2.namedWindow('hw1 video file')  ##預覽視窗的名字
cv2.createTrackbar('progress', 'hw1 video file', 0, total_frame - 1, set_frame_number)  #設定進度條(滑桿)的名字，預設值，最小值，最大值，設定滑桿的值

# 取得YouTube上的影片

In [3]:
import pafy
import time

#設定要播放的Youtube影片
url = "https://youtu.be/PHqhEgkGfrs"
ty_video = pafy.new(url, basic=False, gdata=False)
best = ty_video.getbest(preftype="mp4")
youtube_video = cv2.VideoCapture(best.url)


# 平均濾波

In [4]:
def mean_filter(image, kernel_size):
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size * kernel_size)
    return cv2.filter2D(image, -1, kernel)

# 傅立葉高通濾波

In [5]:
def fft_high_pass_filter(image, freq_cutoff):
    fft_image = np.fft.fft2(image.copy())
    fft_image = np.fft.fftshift(fft_image)
    cy, cx = fft_image.shape[0]/2,fft_image.shape[1]/2
    h = np.arange(fft_image.shape[0]).reshape((-1,1)) - cy
    w = np.arange(fft_image.shape[1]).reshape((1,-1)) - cx
    #freq_cutoff = freq_cutoff**2
    fft_image[:,:,0] =  np.where((h**2)/((freq_cutoff)**2)+w**2/((0.2*freq_cutoff)**2)>=1,fft_image[:,:,0],0)
    fft_image[:,:,1] =  np.where((h**2)/((freq_cutoff)**2)+w**2/((0.2*freq_cutoff)**2)>=1,fft_image[:,:,1],0)
    fft_image[:,:,2] =  np.where((h**2)/((freq_cutoff)**2)+w**2/((0.2*freq_cutoff)**2)>=1,fft_image[:,:,2],0)
    fft_image = np.fft.ifftshift(fft_image)
    fft_image = np.fft.ifft2(fft_image)
    fft_image = np.abs(fft_image)
    return fft_image


# 利用TENSORFLOW做卷積

In [6]:
import tensorflow as tf
def tf_conv_BGRToYIQ(image):
    kernel = np.zeros((1,1,3,3), np.float32)
    kernel[0,0,:,:] = np.array([[0.3, 0.59, 0.11],
                                [0.6, -0.28, -0.32],
                                [0.21, -0.52, 0.31]])
    kernel = tf.constant(kernel, dtype=tf.float32)
    image = tf.constant(np.expand_dims(image,axis=0), dtype=tf.float32)
    return tf.nn.conv2d(image, kernel, strides=[1, 1], padding='SAME').numpy()

def tf_conv_YIQtoBGR(image):
    kernel = np.zeros((1,1,3,3), np.float32)
    kernel[0,0,:,:] = np.array([[1, 0.95, 0.62],
                                [1, -0.28, -0.64],
                                [1, -1.1, 1.72]])
    kernel = tf.constant(kernel, dtype=tf.float32)
    image = tf.constant(np.expand_dims(image,axis=0), dtype=tf.float32)
    return tf.nn.conv2d(image, kernel, strides=[1, 1], padding='SAME').numpy()


In [7]:

while frame_num < total_frame:
    cv2.setTrackbarPos('progress', 'hw1 video file', frame_num)  #設定進度條的值
    hw1_input_video.set(cv2.CAP_PROP_POS_FRAMES, frame_num)  #設定要讀取的frame
    youtube_video.set(cv2.CAP_PROP_POS_FRAMES, frame_num)  #設定要讀取的frame

    ret, frame = hw1_input_video.read()  #讀取那個frame   frame是 高*宽*3的矩阵  #ret是布林值，是否有讀到影片的下一幀
    ret_yt, frame_yt = youtube_video.read()  #讀取那個frame   frame是 高*宽*3的矩阵  #ret是布林值，是否有讀到影片的下一幀

    output_frame = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]), dtype=np.uint8) #設定一個空的矩陣(輸出用)



    #resize輸入維度是 寬*高
    ytFrame_resize = cv2.resize(frame_yt, (int(frame.shape[1] / 2), int(frame.shape[0] / 2))) #Youtube影片frame的縮放
    one_fourth_frame = cv2.resize(frame, (int(frame.shape[1] / 2), int(frame.shape[0] / 2)))  #輸入影片的縮放

    mean_filtered_frame = mean_filter(one_fourth_frame, 20)  #做平均濾波
    high_pass_filtered_frame = fft_high_pass_filter(one_fourth_frame, 272)  #做傅立葉高通濾波
    hsv_frame = cv2.cvtColor(one_fourth_frame, cv2.COLOR_BGR2HSV) #色彩空間轉換
    hsv_frame[:,:,1] = hsv_frame[:,:,1]*1.5
    hsv_frame =  cv2.cvtColor(one_fourth_frame, cv2.COLOR_HSV2BGR).clip(0,255)

    YIQ_frame = tf_conv_BGRToYIQ(ytFrame_resize)
    YIQ_frame[:,:,:,1] = YIQ_frame[:,:,:,1] * 0
    YIQ_frame[:,:,:,2] = YIQ_frame[:,:,:,2] * 0
    YIQ_frame = tf_conv_YIQtoBGR(YIQ_frame)
    YIQ_frame = np.squeeze(YIQ_frame).astype(int).clip(0,255)

    output_frame[0:int(frame.shape[0] / 2), 0:int(frame.shape[1] / 2), :] = YIQ_frame  #左上角放Youtube影片
    output_frame[0:int(frame.shape[0] / 2), int(frame.shape[1] / 2):frame.shape[1], :] = hsv_frame
    output_frame[int(frame.shape[0] / 2):frame.shape[0], 0:int(frame.shape[1] / 2), :] = mean_filtered_frame
    output_frame[int(frame.shape[0] / 2):frame.shape[0], int(frame.shape[1] / 2):frame.shape[1],
    :] = high_pass_filtered_frame
    if ret == False:
        break
    cv2.putText(img=output_frame, text='YIQ without IQ(using tensorflow)', org=(10, 60), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 255, 0),thickness=1)
    cv2.putText(img=output_frame, text='average filtering', org=(10, 1030), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 255, 0),thickness=1)
    cv2.putText(img=output_frame, text='HSV with higher Saturation', org=(1430, 60), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(200, 18, 255),thickness=1)
    cv2.putText(img=output_frame, text='fourier high pass filtering', org=(1450, 1030), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 255, 0),thickness=1)

    previewFrame = cv2.resize(output_frame, (1280, 720))
    cv2.imshow('hw1 video file', previewFrame)
    key = cv2.waitKey(5) & 0xFF
    if key == 27:
        break
    frame_num += 1
    hw1_output_video.write(output_frame)
hw1_input_video.release()
hw1_output_video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1